In [ ]:
import math
from PIL import Image
import io
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, box
import json
import pickle
import time
import os

class GeoDataCollector:
    def __init__(self, latitude_center, longitude_center, grid_rows=50, grid_cols=20,
                 zoom_level=17, distance=450, checkpoint_file='ccontrolprogress_checkpoint.pkl'):
        """
        Initialize the GeoDataCollector with specified parameters
        
        Args:
            latitude_center (float): Center latitude of the area
            longitude_center (float): Center longitude of the area
            grid_rows (int): Number of rows in the grid
            grid_cols (int): Number of columns in the grid
            zoom_level (int): Zoom level for maps
            distance (int): Coverage distance in meters (450m)
            checkpoint_file (str): File to save progress
        """
        # Add checkpoint file path to restore progress
        self.checkpoint_file = checkpoint_file
        # Initialize metrics dictionary to store all processed metrics
        self.metrics_data = {}
        
        # Restore progress from last checkpoint
        self.load_checkpoint()
        
        self.latitude_center = round(latitude_center, 4)
        self.longitude_center = round(longitude_center, 4)
        self.grid_rows = grid_rows
        self.grid_cols = grid_cols
        
        # Calculate offsets for 450m grid cells
        self.lat_offset = 0.004  # Approximately 450m in latitude
        # Adjust longitude offset based on latitude (due to Earth's curvature)
        self.lon_offset = self.lat_offset / math.cos(math.radians(latitude_center))
        
        self.zoom_level = zoom_level
        self.distance = distance

        # Create directory for maps
        self.directory = 'control_mapsgridc180'
        os.makedirs(self.directory, exist_ok=True)
        
        # Configure OSM
        #ox.config(overpass_endpoint="https://lz4.overpass-api.de/api/interpreter")
        
        # Calculate the overall bounds for the entire area
        self.overall_bounds = self.calculate_overall_bounds()
        
        # Store for the entire area's data
        self.area_geometries = None

    def calculate_overall_bounds(self):
        """
        Calculate the bounds for the entire area covering all grid cells
        
        Returns:
            dict: Overall bounds containing north, south, east, and west coordinates
        """
        # Calculate the maximum extents
        max_lat_offset = (self.grid_rows / 2) * self.lat_offset
        max_lon_offset = (self.grid_cols / 2) * self.lon_offset
        
        bounds = {
            'north': round(self.latitude_center + max_lat_offset, 6),
            'south': round(self.latitude_center - max_lat_offset, 6),
            'east': round(self.longitude_center + max_lon_offset, 6),
            'west': round(self.longitude_center - max_lon_offset, 6)
        }
        
        print(f"Overall area bounds: {bounds}")
        return bounds

    def load_checkpoint(self):
        """
        Restore progress from the last saved checkpoint
        """
        if os.path.exists(self.checkpoint_file):
            try:
                with open(self.checkpoint_file, 'rb') as f:
                    checkpoint = pickle.load(f)
                    self.last_processed_idx = checkpoint.get('last_processed_idx', -1)
                    self.metrics_data = checkpoint.get('metrics_data', {})
                    print(f"Restored from checkpoint, last processed index: {self.last_processed_idx}")
                    print(f"Restored metrics data for {len(self.metrics_data)} grid cells")
            except Exception as e:
                print(f"Failed to load checkpoint: {e}")
                self.last_processed_idx = -1
                self.metrics_data = {}
        else:
            self.last_processed_idx = -1
            self.metrics_data = {}

    def save_checkpoint(self, current_idx):
        """
        Save current processing progress
        
        Args:
            current_idx (int): Current processing index
        """
        try:
            checkpoint = {
                'last_processed_idx': current_idx,
                'metrics_data': self.metrics_data
            }
            with open(self.checkpoint_file, 'wb') as f:
                pickle.dump(checkpoint, f)
            print(f"Saved checkpoint, processed up to index: {current_idx}")
        except Exception as e:
            print(f"Failed to save checkpoint: {e}")

    def get_tile_bounds(self, lat, lon):
        """
        Calculate bounds for a tile
        
        Args:
            lat (float): Latitude of tile center
            lon (float): Longitude of tile center
            
        Returns:
            dict: Bounds containing north, south, east, and west coordinates
        """
        lat_offset = self.lat_offset / 2  # Half offset for each direction
        lon_offset = self.lon_offset / 2

        bounds = {
            'north': round(lat + lat_offset, 6),
            'south': round(lat - lat_offset, 6),
            'east': round(lon + lon_offset, 6),
            'west': round(lon - lon_offset, 6)
        }
        return bounds
    
    def fetch_entire_area_data(self):
        """
        Fetch data for the entire area at once
        
        Returns:
            GeoDataFrame: Geometries for the entire area
        """
        print("Fetching data for the entire area...")
        start_time = time.time()
        
        # Define all the tags we need to fetch at once
        tags = {
            'natural': ['water', 'wetland'],
            'railway': ['rail', 'subway', 'tram', 'light_rail'],
            'highway': [
                'motorway', 'trunk', 'primary', 'secondary', 'tertiary',
                'motorway_link', 'trunk_link', 'primary_link', 'secondary_link', 'tertiary_link',
                'busway'
            ]
        }
        
        try:
            # Fetch all geometries at once for the entire area
            geometries = ox.geometries_from_bbox(
                self.overall_bounds['north'], self.overall_bounds['south'],
                self.overall_bounds['east'], self.overall_bounds['west'],
                tags=tags
            )
            
            if geometries.empty:
                print("No data available for the entire specified region")
                return None
            
            end_time = time.time()
            print(f"Successfully fetched data for the entire area in {end_time - start_time:.2f} seconds")
            print(f"Retrieved {len(geometries)} features")
            
            return geometries
            
        except Exception as e:
            print(f"Error fetching data for the entire area: {str(e)}")
            return None
    
    def create_empty_map(self, lat, lon, bounds, idx):
        """
        Create an empty map when no data is available
        
        Args:
            lat (float): Latitude
            lon (float): Longitude
            bounds (dict): Map bounds
            idx (int): Grid cell index
            
        Returns:
            str: Path to the saved empty map
        """
        try:
            fig, ax = plt.subplots(figsize=(10, 10))
            
            # Set plot parameters
            ax.set_xlim(bounds['west'], bounds['east'])
            ax.set_ylim(bounds['south'], bounds['north'])
            ax.set_axis_off()
            ax.set_facecolor('white')
            fig.patch.set_facecolor('white')

            # Save the empty map
            output_path = os.path.join(
                self.directory,
                f"control_{idx}_{lat}_{lon}.png"
            )
            plt.savefig(output_path, dpi=300, bbox_inches='tight', pad_inches=0,
                    facecolor='white')
            plt.close()
            print(f"Created empty map for grid cell {idx} at ({lat}, {lon})")
            return output_path
        except Exception as e:
            print(f"Error creating empty map: {str(e)}")
            return None
    
    def create_grid_map(self, lat, lon, bounds, idx, geometries):
        """
        Create a map for a specific grid cell using the pre-fetched geometries

        Args:
            lat (float): Latitude of grid center
            lon (float): Longitude of grid center
            bounds (dict): Grid cell bounds
            idx (int): Grid cell index
            geometries (GeoDataFrame): Pre-fetched geometries for the entire area

        Returns:
            str: Path to the saved map
        """
        try:
            print(f"\n--- Processing grid {idx} at ({lat}, {lon}) ---")
            print(f"Grid bounds: {bounds}")

            fig, ax = plt.subplots(figsize=(10, 10))

            # Color map for water features
            color_map = {
                'wetland': '#A6E6E6',
                'water': '#B3D9FF'
            }

            # Create a bounding box for the current grid cell
            grid_box = box(bounds['west'], bounds['south'], bounds['east'], bounds['north'])
            print(f"Grid box: {grid_box}")

            # Check if we have any data
            if geometries is None or geometries.empty:
                print("No geometries data available")
                return self.create_empty_map(lat, lon, bounds, idx)

            print(f"Total geometries before filtering: {len(geometries)}")
            if hasattr(geometries, 'crs'):
                print(f"Geometries CRS: {geometries.crs}")

            # Create a spatial filter
            grid_cell_geometries = geometries[geometries.geometry.intersects(grid_box)]
            print(f"Geometries after filtering: {len(grid_cell_geometries)}")

            if grid_cell_geometries.empty:
                print(f"No data available for grid cell {idx} after filtering")
                return self.create_empty_map(lat, lon, bounds, idx)

            # Process water bodies
            if 'natural' in grid_cell_geometries.columns:
                water_bodies = grid_cell_geometries[grid_cell_geometries['natural'] == 'water']
                print(f"Water bodies found: {len(water_bodies) if not water_bodies.empty else 0}")
                if not water_bodies.empty:
                    water_bodies.plot(
                        ax=ax,
                        facecolor=color_map['water'],
                        edgecolor='none',
                        alpha=0.7
                    )
                    print("Water bodies plotted")

                # Process wetlands
                wetlands = grid_cell_geometries[grid_cell_geometries['natural'] == 'wetland']
                print(f"Wetlands found: {len(wetlands) if not wetlands.empty else 0}")
                if not wetlands.empty:
                    wetlands.plot(
                        ax=ax,
                        facecolor=color_map['wetland'],
                        edgecolor='none',
                        alpha=0.7
                    )
                    print("Wetlands plotted")

            # Process railways
            if 'railway' in grid_cell_geometries.columns:
                railways = grid_cell_geometries[~grid_cell_geometries['railway'].isna()]
                print(f"Railways found: {len(railways) if not railways.empty else 0}")

                # Plot railways as lines
                line_railways = railways[railways.geometry.type.isin(['LineString', 'MultiLineString'])]
                if not line_railways.empty:
                    line_railways.plot(
                        ax=ax,
                        color="#000000",
                        linewidth=2.4,
                        alpha=0.9
                    )
                    print("Railways plotted")

            # Process roads
            if 'highway' in grid_cell_geometries.columns:
                roads = grid_cell_geometries[~grid_cell_geometries['highway'].isna()]
                print(f"Roads found: {len(roads) if not roads.empty else 0}")

                # Plot roads as lines
                line_roads = roads[roads.geometry.type.isin(['LineString', 'MultiLineString'])]
                if not line_roads.empty:
                    line_roads.plot(
                        ax=ax,
                        color="#808080",
                        linewidth=1.2,
                        alpha=0.9
                    )
                    print("Roads plotted")

            # Set plot parameters
            ax.set_xlim(bounds['west'], bounds['east'])
            ax.set_ylim(bounds['south'], bounds['north'])
            ax.set_axis_off()
            ax.set_facecolor('white')
            fig.patch.set_facecolor('white')

            # Save the map
            output_path = os.path.join(
                self.directory,
                f"control_{idx}_{lat}_{lon}.png"
            )
            plt.savefig(output_path, dpi=300, bbox_inches='tight', pad_inches=0,
                        facecolor='white')
            plt.close()
            print(f"Map saved to {output_path}")
            return output_path
        except Exception as e:
            print(f"Error creating grid map: {str(e)}")
            import traceback
            traceback.print_exc()
            return self.create_empty_map(lat, lon, bounds, idx)


    def process_grid_cell(self, lat, lon, idx, geometries):
        """
        Process a single grid cell using pre-fetched geometries
        
        Args:
            lat (float): Latitude
            lon (float): Longitude
            idx (int): Grid cell index
            geometries (GeoDataFrame): Pre-fetched geometries
            
        Returns:
            dict: Processed cell data
        """
        lat = round(lat, 6)
        lon = round(lon, 6)
        
        # Check for cached data
        cell_key = f"{idx}_{lat}_{lon}"
        if cell_key in self.metrics_data:
            print(f"Using cached data for grid cell {idx} at ({lat}, {lon})")
            osm_path = os.path.join(
                self.directory,
                f"control_{idx}_{lat}_{lon}.png"
            )
            return {
                'idx': idx,
                'lat': lat,
                'lon': lon,
                'osm_path': osm_path
            }
        
        print(f"Processing grid cell {idx} at ({lat}, {lon})")
        bounds = self.get_tile_bounds(lat, lon)
        osm_path = self.create_grid_map(lat, lon, bounds, idx, geometries)
        
        # Store data
        self.metrics_data[cell_key] = {
            'idx': idx,
            'lat': lat,
            'lon': lon
        }
        # Save checkpoint
        self.save_checkpoint(idx)
        
        return {
            'idx': idx,
            'lat': lat,
            'lon': lon,
            'osm_path': osm_path
        }

    def process_all_cells(self):
        """
        Process all grid cells in the specified area sequentially
        
        Returns:
            list: Results for all processed cells
        """
        # First, fetch data for the entire area
        start_time = time.time()
        area_geometries = self.fetch_entire_area_data()
        if area_geometries is None:
            print("Failed to fetch data for the entire area. Creating empty maps for all cells.")
        
        # Generate all cell coordinates and indices
        cells_to_process = []
        for i in range(self.grid_rows):
            for j in range(self.grid_cols):
                idx = i * self.grid_cols + j
                lat = round(self.latitude_center + self.lat_offset * (i - self.grid_rows/2), 6)
                lon = round(self.longitude_center + self.lon_offset * (j - self.grid_cols/2), 6)
                
                # Skip already processed cells if restarting
                cell_key = f"{idx}_{lat}_{lon}"
                if idx <= self.last_processed_idx and cell_key in self.metrics_data:
                    continue
                
                cells_to_process.append((lat, lon, idx))
        
        # Process cells sequentially
        results = []
        
        # First add already processed cells from checkpoint
        for i in range(self.grid_rows):
            for j in range(self.grid_cols):
                idx = i * self.grid_cols + j
                lat = round(self.latitude_center + self.lat_offset * (i - self.grid_rows/2), 6)
                lon = round(self.longitude_center + self.lon_offset * (j - self.grid_cols/2), 6)
                
                cell_key = f"{idx}_{lat}_{lon}"
                if idx <= self.last_processed_idx and cell_key in self.metrics_data:
                    osm_path = os.path.join(
                        self.directory,
                        f"control_{idx}_{lat}_{lon}.png"
                    )
                    
                    result = {
                        'idx': idx,
                        'lat': lat,
                        'lon': lon,
                        'osm_path': osm_path
                    }
                    results.append(result)
                    print(f"Loaded result for grid cell {idx} from checkpoint")
        
        # Process remaining cells sequentially
        if cells_to_process:
            print(f"Processing {len(cells_to_process)} remaining grid cells sequentially")
            for lat, lon, idx in cells_to_process:
                try:
                    result = self.process_grid_cell(lat, lon, idx, area_geometries)
                    results.append(result)
                    print(f"Completed grid cell {idx} at ({lat}, {lon})")
                except Exception as e:
                    print(f"Error processing grid cell {idx} at ({lat}, {lon}): {str(e)}")
        
        # Sort results by index for consistency
        results.sort(key=lambda x: x['idx'])
        
        end_time = time.time()
        total_time = end_time - start_time
        print(f"Total processing time: {total_time:.2f} seconds")
        print(f"Average time per cell: {total_time / (self.grid_rows * self.grid_cols):.2f} seconds")
        
        return results

def main():
    """
    Main execution function
    """
    # Chicago coordinates
    latitude_center = 41.8015    # Chicago center latitude
    longitude_center = -87.8000 
    collector = GeoDataCollector(
        latitude_center=latitude_center,
        longitude_center=longitude_center,
        grid_rows=180,
        grid_cols=120,
        zoom_level=17,
        distance=450
    )

    results = collector.process_all_cells()
    print("Data collection complete!")
    print(f"Processed {len(results)} grid cells")

if __name__ == "__main__":
    main()

In [ ]:
import os
import request
import math
from PIL import Image
import io
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, box
import json
import pickle

class GeoDataCollector:
    def __init__(self, latitude_center, longitude_center, grid_rows=150, grid_cols=150,
                 image_size='512x512', zoom_level=17, distance=450, checkpoint_file='3orlprogress_checkpoint.pkl'):
        """
        Initialize the GeoDataCollector with specified parameters
        
        Args:
            latitude_center (float): Center latitude of the area
            longitude_center (float): Center longitude of the area
            grid_rows (int): Number of rows in the grid
            grid_cols (int): Number of columns in the grid
            image_size (str): Size of output images in pixels
            zoom_level (int): Zoom level for satellite imagery (17 for 450m coverage)
            distance (int): Coverage distance in meters (450m)
            checkpoint_file (str): File to save progress
        """
        # Add checkpoint file path to restore progress
        self.checkpoint_file = checkpoint_file
        # Initialize metrics dictionary to store all processed metrics
        self.metrics_data = {}
        # Restore progress from last checkpoint
        self.load_checkpoint()
        
        self.latitude_center = round(latitude_center, 4)
        self.longitude_center = round(longitude_center, 4)
        self.grid_rows = grid_rows
        self.grid_cols = grid_cols
        
        # Calculate offsets for 450m grid cells
        self.lat_offset = 0.004  # Approximately 450m in latitude
        # Adjust longitude offset based on latitude (due to Earth's curvature)
        self.lon_offset = self.lat_offset / math.cos(math.radians(latitude_center))
        
        self.image_size = image_size
        self.zoom_level = zoom_level
        self.distance = distance

        # Create necessary directories
        self.directories = {
            'satellite': 'testsatellite_imagesgrido150',
            'osm_maps': 'testosm_mapsgrido150',
            'metrics': 'testmetrics_datagrido150'
        }
        for directory in self.directories.values():
            os.makedirs(directory, exist_ok=True)
    def load_checkpoint(self):
        """
        Restore progress from the last saved checkpoint including metrics data
        """
        if os.path.exists(self.checkpoint_file):
            try:
                with open(self.checkpoint_file, 'rb') as f:
                    checkpoint = pickle.load(f)
                    self.last_processed_idx = checkpoint.get('last_processed_idx', -1)
                    self.metrics_data = checkpoint.get('metrics_data', {})
                    print(f"Restored from checkpoint, last processed index: {self.last_processed_idx}")
                    print(f"Restored metrics data for {len(self.metrics_data)} grid cells")
            except Exception as e:
                print(f"Failed to load checkpoint: {e}")
                self.last_processed_idx = -1
                self.metrics_data = {}
        else:
            self.last_processed_idx = -1
            self.metrics_data = {}

    def save_checkpoint(self, current_idx):
        """
        Save current processing progress and metrics data
        
        Args:
            current_idx (int): Current processing index
        """
        try:
            checkpoint = {
                'last_processed_idx': current_idx,
                'metrics_data': self.metrics_data
            }
            with open(self.checkpoint_file, 'wb') as f:
                pickle.dump(checkpoint, f)
            print(f"Saved checkpoint, processed up to index: {current_idx}")
            print(f"Saved metrics data for {len(self.metrics_data)} grid cells")
        except Exception as e:
            print(f"Failed to save checkpoint: {e}")

    def get_tile_bounds(self, lat, lon):
        """
        Calculate bounds for a 450m x 450m tile
        
        Args:
            lat (float): Latitude of tile center
            lon (float): Longitude of tile center
            
        Returns:
            dict: Bounds containing north, south, east, and west coordinates
        """
        # Calculate bounds for exactly 450m coverage
        lat_offset = self.lat_offset / 2  # Half offset for each direction
        lon_offset = self.lon_offset / 2

        bounds = {
            'north': round(lat + lat_offset, 6),
            'south': round(lat - lat_offset, 6),
            'east': round(lon + lon_offset, 6),
            'west': round(lon - lon_offset, 6)
        }
        return bounds

    def download_satellite_image(self, lat, lon, idx, mapbox_token):
        """
        Download satellite image for specified coordinates
        
        Args:
            lat (float): Latitude
            lon (float): Longitude
            idx (int): Grid cell index
            mapbox_token (str): Mapbox API token
            
        Returns:
            tuple: (image_path, bounds)
        """
        bounds = self.get_tile_bounds(lat, lon)
        
        # Calculate the bounding box string for Mapbox
        bbox = f"[{bounds['west']},{bounds['south']},{bounds['east']},{bounds['north']}]"
        
        # Construct Mapbox Static Images API URL
        width, height = map(int, self.image_size.split('x'))
        url = (
            f"https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/"
            f"{bbox}/"
            f"{336}x{336}"
            f"@2x?access_token={mapbox_token}"  # Added @2x for higher resolution
        )

        try:
            response = requests.get(url)
            response.raise_for_status()

            image = Image.open(io.BytesIO(response.content))
            output_path = os.path.join(
                self.directories['satellite'],
                f"satellite_{idx}_{round(lat, 6)}_{round(lon, 6)}.png"
            )
            image.save(output_path)
            return output_path, bounds
        except Exception as e:
            print(f"Satellite image download error: {str(e)}")
            return None, None
    def create_osm_map(self, lat, lon, bounds, idx):
        """
        Create OpenStreetMap visualization for the specified area
        
        Args:
            lat (float): Latitude
            lon (float): Longitude
            bounds (dict): Area bounds
            idx (int): Grid cell index
            
        Returns:
            str: Path to saved map image
        """
        try:
            fig, ax = plt.subplots(figsize=(10, 10))

            # Define land use categories and their colors
            landuse_tags = {
                'landuse': [
                    'residential', 'commercial', 'industrial', 'retail',
                    'farmland', 'forest', 'grass', 'meadow', 'orchard',
                    'vineyard', 'cemetery', 'recreation_ground'
                ],
                'leisure': [
                    'park', 'garden', 'golf_course', 'sports_centre',
                    'stadium', 'pitch', 'playground'
                ],
                'natural': [
                    'wood', 'grassland', 'heath', 'scrub', 'wetland',
                    'water', 'beach', 'sand'
                ],
                'amenity': [
                    'school', 'university', 'hospital', 'parking'
                ]
            }

            # Color scheme for different categories
            color_map = {
                # Urban areas
                'residential': '#FFCA65',
                'commercial': '#FF8080',
                'industrial': '#BFBFBF',
                'retail': '#FF5555',
                'parking': '#EFEFEF',

                # Educational and healthcare
                'school': '#FFF0AA',
                'university': '#FFF0AA',
                'hospital': '#FFE6E6',

                # Green spaces
                'park': '#B2D8B2',
                'garden': '#B2D8B2',
                'recreation_ground': '#B8E6B8',
                'playground': '#CCE6CC',
                'sports_centre': '#CCE6CC',
                'stadium': '#CCE6CC',
                'pitch': '#CCE6CC',
                'golf_course': '#B2D8B2',

                # Natural areas
                'forest': '#8CBF8C',
                'wood': '#8CBF8C',
                'grass': '#CCFFCC',
                'grassland': '#CCFFCC',
                'meadow': '#CCFFCC',
                'heath': '#CCE6CC',
                'scrub': '#B8E6B8',
                'wetland': '#A6E6E6',
                'water': '#B3D9FF',
                'beach': '#FFF5CC',
                'sand': '#FFF5CC',

                # Agricultural
                'farmland': '#FFFFCC',
                'orchard': '#E6FFB3',
                'vineyard': '#E6FFB3',

                # Other
                'cemetery': '#D1CFCD'
            }

            # Download and plot each land use category
            for category, values in landuse_tags.items():
                tags = {category: values}
                try:
                    features = ox.geometries_from_bbox(
                        bounds['north'], bounds['south'],
                        bounds['east'], bounds['west'],
                        tags=tags
                    )

                    if not features.empty:
                        for _, row in features.iterrows():
                            feature_type = row[category] if category in row else None
                            if feature_type in color_map:
                                features.loc[[row.name]].plot(
                                    ax=ax,
                                    facecolor=color_map[feature_type],
                                    edgecolor='grey',
                                    alpha=0.7,
                                    linewidth=0.5
                                )

                except Exception as e:
                    print(f"No {category} data available: {str(e)}")

           

            # Set plot parameters
            ax.set_xlim(bounds['west'], bounds['east'])
            ax.set_ylim(bounds['south'], bounds['north'])
            ax.set_axis_off()
            ax.set_facecolor('white')
            fig.patch.set_facecolor('white')

            # Save the map
            output_path = os.path.join(
                self.directories['osm_maps'],
                f"osm_map_{idx}_{round(lat, 6)}_{round(lon, 6)}.png"
            )
            plt.savefig(output_path, dpi=300, bbox_inches='tight', pad_inches=0,
                       facecolor='white')
            plt.close()
            return output_path
        except Exception as e:
            print(f"Error creating OSM map: {str(e)}")
            return None
    def calculate_metrics(self, lat, lon, bounds):
        """
        Calculate comprehensive urban metrics including building patterns,
        road networks, land use, and accessibility
        
        Args:
            lat (float): Latitude
            lon (float): Longitude
            bounds (dict): Area bounds
            
        Returns:
            dict: Calculated metrics
        """
        try:
            # Create boundary box and convert to proper projection
            bbox = box(bounds['west'], bounds['south'], bounds['east'], bounds['north'])
            bbox_gdf = gpd.GeoDataFrame(geometry=[bbox], crs='epsg:4326')
            bbox_projected = bbox_gdf.to_crs('epsg:3857')
            area_m2 = bbox_projected.geometry.area.iloc[0]
            area_km2 = area_m2 / 1_000_000

            # Get buildings data
            buildings = ox.geometries_from_bbox(
                bounds['north'], bounds['south'],
                bounds['east'], bounds['west'],
                tags={'building': True}
            )

            # Get road network - with robust error handling
            roads = None
            try:
                roads = ox.graph_from_bbox(
                    bounds['north'], bounds['south'],
                    bounds['east'], bounds['west'],
                    network_type='all',
                    simplify=True
                )
                # Verify the graph has nodes and edges
                if roads is None or len(roads.nodes) == 0 or len(roads.edges) == 0:
                    print("Road network is empty or null")
                    roads = None
            except Exception as e:
                print(f"Error creating road network: {str(e)}")
                roads = None

            # Get land use data
            land_use_tags = {
                'landuse': ['residential', 'commercial', 'industrial', 'retail',
                            'recreation', 'institutional'],
                'amenity': ['school', 'hospital', 'park', 'university',
                            'library', 'marketplace'],
                'leisure': ['park', 'garden', 'playground']
            }
            land_uses = ox.geometries_from_bbox(
                bounds['north'], bounds['south'],
                bounds['east'], bounds['west'],
                tags=land_use_tags
            )

            # Calculate building metrics
            building_metrics = {}
            if not buildings.empty:
                buildings = buildings.to_crs('epsg:3857')
                building_area = buildings.geometry.area.sum()
                # Modified: Building density is the ratio of building area to total area
                building_metrics = {
                    'building_density': building_area / area_m2,  # Building coverage ratio
                    'building_footprint_ratio': building_area / area_m2,
                    'avg_building_size': buildings.geometry.area.mean()
                }
            else:
                building_metrics = {
                    'building_density': 0,
                    'building_footprint_ratio': 0,
                    'avg_building_size': 0
                }

            # Calculate road network metrics
            road_metrics = {
                'road_density_km': 0,
                'intersection_density': 0,
                'road_network_complexity': 0
            }
            
            if roads is not None:
                try:
                    # Extract edges and nodes
                    edges = ox.graph_to_gdfs(roads, nodes=False)
                    if not edges.empty:
                        edges = edges.to_crs('epsg:3857')
                        total_length = edges.geometry.length.sum()
                        
                        # Count intersections (nodes with degree > 1)
                        intersection_count = 0
                        for _, degree in roads.degree():
                            if degree > 1:
                                intersection_count += 1
                        
                        # Calculate metrics
                        road_density = total_length / 1000 / area_km2 if area_km2 > 0 else 0
                        intersection_density = intersection_count / area_km2 if area_km2 > 0 else 0
                        
                        # Calculate network complexity
                        edge_count = len(roads.edges)
                        node_count = len(roads.nodes)
                        network_complexity = edge_count / node_count if node_count > 0 else 0
                        
                        road_metrics = {
                            'road_density_km': road_density,
                            'intersection_density': intersection_density,
                            'road_network_complexity': network_complexity
                        }
                except Exception as e:
                    print(f"Road metrics calculation error: {str(e)}")

            # Calculate land use metrics
            land_use_metrics = {}
            if not land_uses.empty:
                # Convert geographic data to correct projection for area calculation
                land_uses = land_uses.to_crs('epsg:3857')
                
                # Calculate area for each land use type
                land_use_areas = {}
                land_use_counts = {}
                
                for category in ['landuse', 'amenity', 'leisure']:
                    if category in land_uses.columns:
                        # Group by category and calculate area for each type
                        for land_type, group in land_uses.groupby(category):
                            if land_type and not pd.isna(land_type):
                                key = f"{category}_{land_type}"
                                land_use_counts[key] = len(group)
                                land_use_areas[key] = group.geometry.area.sum()
                
                # Calculate average land use area
                if land_use_areas:
                    avg_land_use_area = sum(land_use_areas.values()) / len(land_use_areas)
                else:
                    avg_land_use_area = 0
                    
                # Calculate Shannon diversity index
                if land_use_counts:
                    total = sum(land_use_counts.values())
                    proportions = [count/total for count in land_use_counts.values()]
                    shannon_diversity = -sum(
                        p * math.log(p) for p in proportions if p > 0
                    )
                else:
                    shannon_diversity = 0

                land_use_metrics = {
                    'land_use_diversity': shannon_diversity,
                    'unique_land_uses': len(land_use_counts),
                    'avg_land_use_area': avg_land_use_area,  # Average land use area (square meters)
                    'avg_land_use_area_km2': avg_land_use_area / 1_000_000,  # Average land use area (square kilometers)
                    'amenity_density': (
                        sum(1 for _ in land_uses['amenity'].dropna()) / area_km2
                        if 'amenity' in land_uses.columns else 0
                    )
                }
            else:
                land_use_metrics = {
                    'land_use_diversity': 0,
                    'unique_land_uses': 0,
                    'avg_land_use_area': 0,
                    'avg_land_use_area_km2': 0,
                    'amenity_density': 0
                }

            # Calculate accessibility metrics
            accessibility_metrics = {}
            if not land_uses.empty and 'amenity' in land_uses.columns:
                amenities = land_uses[land_uses['amenity'].notna()]
                if not amenities.empty:
                    # Project to Web Mercator for accurate distance calculations
                    amenities = amenities.to_crs('epsg:3857')
                    center_gdf = gpd.GeoDataFrame(
                        geometry=[Point(lon, lat)],
                        crs='epsg:4326'
                    ).to_crs('epsg:3857')
                    center_point = center_gdf.geometry.iloc[0]

                    distances = [
                        center_point.distance(point)
                        for point in amenities.geometry
                    ]
                    accessibility_metrics = {
                        'avg_distance_to_amenities': (
                            sum(distances) / len(distances) / 1000 if distances else 0
                        ),  # Convert to kilometers
                        'nearest_amenity_distance': (
                            min(distances) / 1000 if distances else 0
                        )  # Convert to kilometers
                    }
            else:
                accessibility_metrics = {
                    'avg_distance_to_amenities': 0,
                    'nearest_amenity_distance': 0
                }

            # Combine all metrics
            metrics = {
                **building_metrics,
                **road_metrics,
                **land_use_metrics,
                **accessibility_metrics
            }

            return metrics

        except Exception as e:
            print(f"Metrics calculation error: {str(e)}")
            return {
                'building_density': 0,
                'building_footprint_ratio': 0,
                'avg_building_size': 0,
                'road_density_km': 0,
                'intersection_density': 0,
                'road_network_complexity': 0,
                'land_use_diversity': 0,
                'unique_land_uses': 0,
                'avg_land_use_area': 0,
                'avg_land_use_area_km2': 0,
                'amenity_density': 0,
                'avg_distance_to_amenities': 0,
                'nearest_amenity_distance': 0
            }


   



    def process_grid_cell(self, lat, lon, idx, mapbox_token):
        """
        Process a single grid cell
        
        Args:
            lat (float): Latitude
            lon (float): Longitude
            idx (int): Grid cell index
            mapbox_token (str): Mapbox API token
            
        Returns:
            dict: Processed cell data
        """
        lat = round(lat, 6)
        lon = round(lon, 6)
        
        # Check for cached metrics
        cell_key = f"{idx}_{lat}_{lon}"
        if cell_key in self.metrics_data:
            print(f"Using cached metrics for grid cell {idx} at ({lat}, {lon})")
            metrics = self.metrics_data[cell_key]['metrics']
            satellite_path = os.path.join(
                self.directories['satellite'],
                f"satellite_{idx}_{lat}_{lon}.png"
            )
            osm_path = os.path.join(
                self.directories['osm_maps'],
                f"osm_map_{idx}_{lat}_{lon}.png"
            )
        else:
            print(f"Processing grid cell {idx} at ({lat}, {lon})")
            bounds = self.get_tile_bounds(lat, lon)
            satellite_path, _ = self.download_satellite_image(lat, lon, idx, mapbox_token)
            osm_path = self.create_osm_map(lat, lon, bounds, idx)
            metrics = self.calculate_metrics(lat, lon, bounds)
            
            # Store metrics
            self.metrics_data[cell_key] = {
                'idx': idx,
                'lat': lat,
                'lon': lon,
                'metrics': metrics
            }

        # Save checkpoint
        self.save_checkpoint(idx)
        
        return {
            'idx': idx,
            'lat': lat,
            'lon': lon,
            'satellite_path': satellite_path,
            'osm_path': osm_path,
            'metrics': metrics
        }
    
    def process_all_cells(self, mapbox_token):
        """
        Process all grid cells in the specified area
        
        Args:
            mapbox_token (str): Mapbox API token
            
        Returns:
            list: Results for all processed cells
        """
        results = []
        
        # Process each cell in the grid
        for i in range(self.grid_rows):
            for j in range(self.grid_cols):
                idx = i * self.grid_cols + j
                
                # Calculate cell center coordinates
                lat = round(self.latitude_center + self.lat_offset * (i - self.grid_rows/2), 6)
                lon = round(self.longitude_center + self.lon_offset * (j - self.grid_cols/2), 6)
                
                # Skip already processed cells if restarting
                if idx <= self.last_processed_idx:
                    cell_key = f"{idx}_{lat}_{lon}"
                    if cell_key in self.metrics_data:
                        cell_data = self.metrics_data[cell_key]
                        satellite_path = os.path.join(
                            self.directories['satellite'],
                            f"satellite_{idx}_{lat}_{lon}.png"
                        )
                        osm_path = os.path.join(
                            self.directories['osm_maps'],
                            f"osm_map_{idx}_{lat}_{lon}.png"
                        )
                        
                        result = {
                            'idx': idx,
                            'lat': lat,
                            'lon': lon,
                            'satellite_path': satellite_path,
                            'osm_path': osm_path,
                            'metrics': cell_data['metrics']
                        }
                        results.append(result)
                        print(f"Loaded result for grid cell {idx} from checkpoint")
                        continue
                
                result = self.process_grid_cell(lat, lon, idx, mapbox_token)
                results.append(result)

        # Save final results
        self.save_results(results)
        return results

    def save_results(self, results):
        """
        Save processing results to CSV and JSON files
        
        Args:
            results (list): List of processed cell results
        """
        try:
            # Save metrics to CSV
            metrics_data = []
            for r in results:
                if r and r['metrics']:
                    metrics_data.append({
                        'idx': r['idx'],
                        'lat': r['lat'],
                        'lon': r['lon'],
                        **r['metrics']
                    })

            df = pd.DataFrame(metrics_data)
            df.to_csv(os.path.join(self.directories['metrics'], 'metrics_results.csv'), 
                     index=False)
            
            # Save processed file paths
            processed_files = {
                'satellite': [r['satellite_path'] for r in results if r and r['satellite_path']],
                'osm_maps': [r['osm_path'] for r in results if r and r['osm_path']]
            }
            
            with open('processed_files.json', 'w') as f:
                json.dump(processed_files, f)
                
            print(f"Successfully saved results for {len(metrics_data)} cells")
        
        except Exception as e:
            print(f"Error saving results: {e}")

def main():
    """
    Main execution function
    """
    latitude_center = 28.5083    # Orlando center latitude
    longitude_center = -81.3092
    # Mapbox access token
    mapbox_token = 'replace'

    # Initialize collector with rectangular grid (10x15)
    collector = GeoDataCollector(
        latitude_center=latitude_center,
        longitude_center=longitude_center,
        grid_rows=150,
        grid_cols=150,
        zoom_level=17,
        # Adjusted for 450m coverage
        distance=450    # Set to exactly 450m
    )

    # Process all grid cells
    results = collector.process_all_cells(mapbox_token)
    print("Data collection complete!")
    
    # Print summary
    print(f"Processed {len(results)} grid cells")
    print(f"Coverage area: {collector.grid_rows * 450}m x {collector.grid_cols * 450}m")
    print(f"Total area: {(collector.grid_rows * collector.grid_cols * 450 * 450) / 1000000:.2f} km²")

if __name__ == "__main__":
    main()